In [0]:
#Reset Button. Press when memory goes crazy fat !
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
# imports cell
import numpy as np
import _pickle as cPickle
import sys 
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

# Deep Learning libs
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Conv2D,Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint


# Let's start with data

We use DeepSig Dataset: RadioML 2016.04C<br>
A synthetic dataset, generated with GNU Radio, consisting of 11 modulations. This is a
variable-SNR dataset with moderate LO drift, light fading, and numerous different
labeled SNR increments for use in measuring performance across different signal and
noise power scenarios.

## Loading data

**The file is formatted as a "pickle" file which can be open for example in python by using cPickle.load(...)**



In [2]:
#Loading the data from google drive 

from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive"
filename = "/content/drive/My Drive/RML2016.10b.dat"
open_file = open(filename,'rb')
data = cPickle.load(open_file, encoding='latin1')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
'My Drive'


In [0]:
# Use lists when accessing data from dict for ease of access.
keys_list = list(data.keys())
temp_list = []
label_list = []
snr_list = []
for i in range(len(keys_list)):
    curr_item = data[keys_list[i]] 
    temp_list.append(curr_item)
    for j in range(curr_item.shape[0]):
        label_list.append(keys_list[i][0])
    #for j in rane(curr_item.shape[0]):
    #    snr_list.append(keys_list[i][1])

# Convert all lists into numpy arrays.
X = np.array(temp_list).reshape(1200000,2,128)
Y = np.array(label_list)

# Clear All un-neccsarry lists created.
temp_list.clear()
label_list.clear()
snr_list.clear()
keys_list.clear()
data.clear()

## Create feature Spaces for data

**Every sample is presented using two vectors each of them has 128 elements. Those two vectors are the in-phase and quadrature phase components of a sample.**

**In this notebook we only use the raw features data**

## Splitting data

**Split the data into 50% for training/validation and 50% for testing.**

In [0]:
(trainX, testX, trainY, testY) = train_test_split(X,Y, test_size=0.50, random_state=42)

## Encoding Labels

Our class labels are currently represented as strings; however, Keras will assume that both:

1. Labels are encoded as integers

2. And furthermore, one-hot encoding is performed on these labels making each label represented as a vector rather than an integer

To accomplish this encoding, we can use the LabelBinarizer  class from scikit-learn

In [0]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# Defining the Fully Connected Model
<ul>

1. The core data structure of Keras is a model. The simplest type of model is the Sequential model, a linear stack of layers.

2. A 3-layer deep neural network consisiting only of fully connected layers of size 512, 256, 11 neurons.


In [7]:
# Declare layers size
in_shape = (2,128)
hidden1_size = 512
hidden2_size = 256
hidden3_size = 10

# Build model
model = Sequential()
model.add(Dense(hidden1_size, input_shape = in_shape))
model.add(Activation('relu'))
model.add(Dense(hidden2_size))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(hidden3_size))
model.add(Activation('softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2, 512)            66048     
_________________________________________________________________
activation_1 (Activation)    (None, 2, 512)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2, 256)            131328    
_________________________________________________________________
activation_2 (Activation)    (None, 2, 256)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
_________________________________________________________________
acti

## Fitting the model

1. The training process will run for a fixed number of iterations (epochs) through the dataset.

2. We can also set the number of instances that are evaluated before a weight update in the network is performed (batch size).


In [54]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_fc_model_raw.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.fit(trainX, trainY, epochs=150, batch_size=750, validation_split=0.05,callbacks=[es,mc])

Train on 570000 samples, validate on 30000 samples
Epoch 1/150
570000/570000 [==============================] - 4s 7us/step - loss: 0.9009 - acc: 0.6419 - val_loss: 1.5280 - val_acc: 0.4841

Epoch 00001: val_acc improved from -inf to 0.48407, saving model to best_fc_model_raw.h5
Epoch 2/150
570000/570000 [==============================] - 4s 7us/step - loss: 0.9001 - acc: 0.6422 - val_loss: 1.5405 - val_acc: 0.4846

Epoch 00002: val_acc improved from 0.48407 to 0.48460, saving model to best_fc_model_raw.h5
Epoch 3/150
570000/570000 [==============================] - 4s 7us/step - loss: 0.9013 - acc: 0.6422 - val_loss: 1.5340 - val_acc: 0.4822

Epoch 00003: val_acc did not improve from 0.48460
Epoch 4/150
570000/570000 [==============================] - 4s 7us/step - loss: 0.9003 - acc: 0.6420 - val_loss: 1.5358 - val_acc: 0.4833

Epoch 00004: val_acc did not improve from 0.48460
Epoch 5/150
570000/570000 [==============================] - 4s 7us/step - loss: 0.8991 - acc: 0.6424 - val_

# Defining the Convolutional Model
<ul>
    1.  Reshape the input to be the same as the first layer of the network's input

In [20]:
# Declare layers size
in_shape = (2,128)
conv1_kernel_shape=(3,1)
conv1_number_of_filters=64
conv2_kernel_shape=(3,2)
conv2_number_of_filters=16
dense1_size = 128
dense2_size = 10


# Build model
model_conv = Sequential()
model_conv.add(Reshape((128,2,1), input_shape=in_shape))
model_conv.add(Conv2D(conv1_number_of_filters, conv1_kernel_shape, strides=1,
                 padding='same', data_format='channels_last',activation='relu'))
model_conv.add(Conv2D(conv2_number_of_filters, conv2_kernel_shape, strides=1,
                 padding='same', data_format='channels_last',activation='relu'))
model_conv.add(Flatten())
model_conv.add(Dense(dense1_size, activation='relu'))
model_conv.add(Dense(dense2_size, activation='softmax'))

# Compile model
model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 128, 2, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 2, 64)        256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 2, 16)        6160      
_________________________________________________________________
flatten_3 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               524416    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 532,122
Trainable params: 532,122
Non-trainable params: 0
_________________________________________________________________


In [36]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_cnn_model_raw.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model_conv.fit(trainX, trainY, epochs=150, batch_size=30000, validation_split=0.05,callbacks=[es,mc])

Train on 570000 samples, validate on 30000 samples
Epoch 1/150
570000/570000 [==============================] - 10s 17us/step - loss: 1.0043 - acc: 0.5954 - val_loss: 1.1378 - val_acc: 0.5398

Epoch 00001: val_acc improved from -inf to 0.53977, saving model to best_cnn_model_raw.h5
Epoch 2/150
570000/570000 [==============================] - 10s 17us/step - loss: 1.0049 - acc: 0.5952 - val_loss: 1.1386 - val_acc: 0.5398

Epoch 00002: val_acc improved from 0.53977 to 0.53983, saving model to best_cnn_model_raw.h5
Epoch 3/150
570000/570000 [==============================] - 10s 18us/step - loss: 1.0046 - acc: 0.5954 - val_loss: 1.1364 - val_acc: 0.5422

Epoch 00003: val_acc improved from 0.53983 to 0.54217, saving model to best_cnn_model_raw.h5
Epoch 4/150
570000/570000 [==============================] - 10s 18us/step - loss: 1.0037 - acc: 0.5957 - val_loss: 1.1372 - val_acc: 0.5415

Epoch 00004: val_acc did not improve from 0.54217
Epoch 5/150
570000/570000 [============================